In [1]:
import torch
from transformers import logging
from transformers import pipeline
from transformers import BertTokenizer, BertModel


In [2]:
from LamaTRExData import LamaTRExData

In [3]:
TREx = LamaTRExData()
TREx.load()

break

In [4]:
logging.set_verbosity_error()

In [5]:
#model = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-uncased');

In [6]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

In [7]:
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
output = model(**encoded_input)

In [8]:
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("The man worked as a [MASK].")

[{'score': 0.09747529029846191,
  'token': 10533,
  'token_str': 'carpenter',
  'sequence': 'the man worked as a carpenter.'},
 {'score': 0.052383411675691605,
  'token': 15610,
  'token_str': 'waiter',
  'sequence': 'the man worked as a waiter.'},
 {'score': 0.04962699115276337,
  'token': 13362,
  'token_str': 'barber',
  'sequence': 'the man worked as a barber.'},
 {'score': 0.0378858707845211,
  'token': 15893,
  'token_str': 'mechanic',
  'sequence': 'the man worked as a mechanic.'},
 {'score': 0.03768062964081764,
  'token': 18968,
  'token_str': 'salesman',
  'sequence': 'the man worked as a salesman.'}]